In [1]:
%endpoint http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v3-habilitations-partielles-inference

Endpoint set to: http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v3-habilitations-partielles-inference

In [2]:
%endpoint http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v5-inference-legal-base

Endpoint set to: http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v5-inference-legal-base

<center><h1>Movies</h1></center>

---

## Modélisation des habilitations doctorales

### Habilitation simple

<center>
    <figure>
        <img width="90%" src="habilitation_doctorale.svg">
        <figcaption>Modélisation de l'habilitation doctorale</figcaption>
    </figure>
</center>

### Délégation de l'habilitation doctorale

<center>
    <figure>
        <img width="90%" src="delegation_doctorale.svg">
        <figcaption>Modélisation de la délégation doctorale</figcaption>
    </figure>
</center>

__États des établissements__

En matière de compétences doctorales il existe quatre états distincts :

* `habilitation` l'établissement possède l'intégralité de l'habilitation
* `delegationDoctorale` l'établissement délègue l'intégralité de son habilitation à un établissement tiers


Quelques règles régissent les état doctoraux : ils sont exclusifs les uns des autres ; ils bornés chronologiquement, ces bornes doivent s'inscrire dans celles de l'établissement auquel ils se rapportent : une habilitation ne peut pas être antérieure à la création d'un établissement ou postérieure à la fermeture d'un établissement ; l'état de délégation partielle implique nécessairement l'existence d'un état d'habilitation partielle.

__Propriétés :__

| nom        | description             | obligatoire                     | note |
|------------|-------------------------|---------------------------------|------|
| `from`     | établissement source    | <input type="checkbox" checked> |      |
| `to`       | établissement cible     | <input type="checkbox">         |      |
| `start`    | date de début de l'état | <input type="checkbox" checked> |      |
| `end`      | date de fin de l'état   | <input type="checkbox">         |      |
| `notes`    |                         | <input type="checkbox">         |      |



## Cas d'usage

### Liste des habilitations doctorales

> un établissement peut être habilité plusieurs fois

In [3]:
PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        
SELECT DISTINCT ?etat_doctoral ?etat_label ?etab_label ?start ?end ?legal_base WHERE {
    ?etat_doctoral a [ ?x mov:EtatDoctoral; rdfs:label ?etat_label];
                                            mov:from [skos:prefLabel ?etab_label].

    OPTIONAL {
        ?etat_doctoral mov:legalBase ?legal_base.
    }
    
    OPTIONAL {
        ?etat_doctoral mov:start ?start.
    }
    
    OPTIONAL {
        ?etat_doctoral mov:end ?end.
    }
    
    FILTER(LANG(?etat_label) = "en")
    FILTER(LANG(?etab_label) = "fr")
}

etat_doctoral,etat_label,etab_label,start,end,legal_base
http://www.abes.fr/entities/etats/39f0615d482d632921b20eeb172dfe68198911a4,Habilitation,Institut national des sciences et industries du vivant et de l'environnement,2006-12-13,2015-08-31,https://www.legifrance.gouv.fr/loda/id/JORFTEXT000000245929/
http://www.abes.fr/entities/etats/99a58ffd966bfb4d33aca15de76517a1aee0a838,Habilitation,Aix-Marseille I,???,2011-12-31,
http://www.abes.fr/entities/etats/ad6e7eba8c1f6367120899274cefda956175066e,Habilitation,Université d'Aix-Marseille III,???,2011-12-31,https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000000500623
http://www.abes.fr/entities/etats/5e0d6aa15e81887fd6dc73918341c5ca9b2194f9,Habilitation,Aix-Marseille Université,2012-01-01,9999-12-31,https://www.legifrance.gouv.fr/loda/id/JORFTEXT000024502554
http://www.abes.fr/entities/etats/ea165cfce6e5bf6acabb15c6650c1e7a53d54a13,Habilitation,Université d’Amiens,1971-06-29,9999-12-31,https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000000853165
http://www.abes.fr/entities/etats/10ba3ae6650834efe1991ff6c5a56095841e108a,Habilitation,Université d'Angers,1971-10-25,9999-12-31,https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000000336401
http://www.abes.fr/entities/etats/5e4278067716f67dc4e4ce0047960ffc03ed185a,Habilitation,Université des Antilles,2015-06-25,9999-12-31,
http://www.abes.fr/entities/etats/7421f66245519b497f80b8003b40d00d133c48c1,Habilitation,Université d'Artois,1991-11-07,9999-12-31,https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000000355616
http://www.abes.fr/entities/etats/4960024b2f106943aa10981630490c393830b325,Habilitation,Université d'Avignon,1984-07-17,9999-12-31,
http://www.abes.fr/entities/etats/c1fba056a399446ffa43a0ea8fe249a8ec1c2ea4,Habilitation,Université Côte d'Azur,2016-09-01,9999-12-31,


### Liste des établissements habilités

In [4]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        
SELECT DISTINCT ?etat_doctoral ?etat_label ?etab_label ?start ?end WHERE {
    ?etat_doctoral a [ ?x mov:EtatDoctoral; rdfs:label ?etat_label];
        mov:from [skos:prefLabel ?etab_label].
    
    OPTIONAL {
        ?etat_doctoral mov:start ?start.
    }
    
    OPTIONAL {
        ?etat_doctoral mov:end ?end.
    }
    
    FILTER(LANG(?etat_label) = "en")
    FILTER(LANG(?etab_label) = "fr")
    FILTER (?end > NOW())
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

etat_doctoral,etat_label,etab_label,start,end
http://www.abes.fr/entities/etats/5e0d6aa15e81887fd6dc73918341c5ca9b2194f9,Habilitation,Aix-Marseille Université,2012-01-01,9999-12-31
http://www.abes.fr/entities/etats/ea165cfce6e5bf6acabb15c6650c1e7a53d54a13,Habilitation,Université d’Amiens,1971-06-29,9999-12-31
http://www.abes.fr/entities/etats/10ba3ae6650834efe1991ff6c5a56095841e108a,Habilitation,Université d'Angers,1971-10-25,9999-12-31
http://www.abes.fr/entities/etats/5e4278067716f67dc4e4ce0047960ffc03ed185a,Habilitation,Université des Antilles,2015-06-25,9999-12-31
http://www.abes.fr/entities/etats/7421f66245519b497f80b8003b40d00d133c48c1,Habilitation,Université d'Artois,1991-11-07,9999-12-31
http://www.abes.fr/entities/etats/4960024b2f106943aa10981630490c393830b325,Habilitation,Université d'Avignon,1984-07-17,9999-12-31
http://www.abes.fr/entities/etats/c1fba056a399446ffa43a0ea8fe249a8ec1c2ea4,Habilitation,Université Côte d'Azur,2016-09-01,9999-12-31
http://www.abes.fr/entities/etats/a3d019e993f388972cb224720e0cd995e4631879,Habilitation,Université de Bordeaux-III,???,9999-12-31
http://www.abes.fr/entities/etats/c8cf4806a7e7ef6b8cf9a31766665f08a189b34b,Habilitation,Université de Bordeaux,2014-01-01,9999-12-31
http://www.abes.fr/entities/etats/5882cd617cdfca6071feb480c3581715b41d5b14,Habilitation,Université de Brest,1975-08-28,9999-12-31


### Liste des établissements qui ne sont plus habilités

In [5]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        
SELECT DISTINCT ?etat_doctoral ?etat_label ?etab_label ?start ?end WHERE {
    ?etat_doctoral a [ ?x mov:EtatDoctoral; rdfs:label ?etat_label];
        mov:from [skos:prefLabel ?etab_label].
    
    OPTIONAL {
        ?etat_doctoral mov:start ?start.
    }
    
    OPTIONAL {
        ?etat_doctoral mov:end ?end.
    }
    
    FILTER(LANG(?etat_label) = "en")
    FILTER(LANG(?etab_label) = "fr")
    FILTER (?end < NOW())
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

etat_doctoral,etat_label,etab_label,start,end
http://www.abes.fr/entities/etats/39f0615d482d632921b20eeb172dfe68198911a4,Habilitation,Institut national des sciences et industries du vivant et de l'environnement,2006-12-13,2015-08-31
http://www.abes.fr/entities/etats/99a58ffd966bfb4d33aca15de76517a1aee0a838,Habilitation,Aix-Marseille I,???,2011-12-31
http://www.abes.fr/entities/etats/ad6e7eba8c1f6367120899274cefda956175066e,Habilitation,Université d'Aix-Marseille III,???,2011-12-31
http://www.abes.fr/entities/etats/0b4ae3918b7a1cd0cc73760cafa0ed5eaf5e073f,Habilitation,Université de technologie de Belfort-Montbéliard,2000-08-21,2016-12-31
http://www.abes.fr/entities/etats/e7c8f219fafb585be292c8241ac3a2a89ebe44d9,Habilitation,Université de Besançon,1968-01-01,2016-12-31
http://www.abes.fr/entities/etats/6872169f3f515e9d517a182be0d4df94bfcc6308,Habilitation,Université Bordeaux-I,???,2013-12-31
http://www.abes.fr/entities/etats/3262afae56d4e7cdaff11fa343cbcfb3b2c3d922,Habilitation,Université Bordeaux-II,???,2013-12-31
http://www.abes.fr/entities/etats/900fa190254436ccd8c46adbfd56037a844dcff4,Habilitation,Université Bordeaux-IV,???,2013-12-31
http://www.abes.fr/entities/etats/a0c6b696936323e2ab2fa4a2208dce76a4a317e4,Habilitation,Université de Caen,1432-01-01,2016-12-31
http://www.abes.fr/entities/etats/0096cea7247bd98f2cb891e1bf856a13e6448c86,Habilitation,Université de Cergy-Pontoise,1993-01-01,2019-12-31


### Filiation des environnements thèses

* Retrouver les filiation des habilitations doctorales à partir d'un établissement. Permet de regrouper les thèses d'une institution. Par exemple celles de Grenoble Alpes (expérimental) avec celle de Grenoble Alpes (Comue) etc.


In [6]:
%qparam infer true
%qparam sameAs true
%show 20

PREFIX paysage: <http://paysage.mesri.fr/structures/>
PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

select DISTINCT ?etab_label  ?code_etab ?start ?end

    
where {   
    BIND(paysage:Y7ch7 as ?grenobleAlpes)
    
    ?grenobleAlpes (^mov:to/mov:from|^mov:from/mov:to)+ [?x ?etab; skos:prefLabel ?etab_label].
    
    
    ?etat a mov:Existence;
          mov:from ?etab.
    
    ?etab mov:codeEtabid ?code_etab.
    
    OPTIONAL {
        ?delegation a mov:Delegation;
            mov:from ?etab;
            mov:start ?start.
        
        OPTIONAL {
            ?delegation mov:end ?end.
        }
    }
    
    FILTER(LANG(?etab_label) = "fr")
}

Param set: infer = true
Param set: sameAs = true
Result maximum size: 20

etab_label,code_etab,start,end
Université Grenoble Alpes,GREA,,
Université Grenoble I,GRE1,,
Université Grenoble-II,GRE2,,
Université Grenoble-III,GRE3,,
Université de Chambéry,CHAM,2010-11-01,2014-12-31
Université de Chambéry,CHAM,2015-01-01,2019-08-31
Institut polytechnique de Grenoble,INPG,2010-11-01,2014-12-31


In [7]:
%qparam infer true
%qparam sameAs true
%show 20

PREFIX paysage: <http://paysage.mesri.fr/structures/>
PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

select DISTINCT ?etab_label  ?code_etab ?start ?end

    
where {   
    BIND(paysage:dV766 as ?saclay)
    
    ?saclay (^mov:to/mov:from|^mov:from/mov:to)+ [?x ?etab; skos:prefLabel ?etab_label].
    
    ?etat a mov:Existence;
          mov:from ?etab.
    
    ?etab mov:codeEtabid ?code_etab.
    
    OPTIONAL {
        ?delegation a mov:Delegation;
            mov:from ?etab;
            mov:start ?start.
        
        OPTIONAL {
            ?delegation mov:end ?end.
        }
    }
    
    FILTER(LANG(?etab_label) = "fr")
}

Param set: infer = true
Param set: sameAs = true
Result maximum size: 20

etab_label,code_etab,start,end
Institut national des sciences et industries du vivant et de l'environnement,AGPT,2015-09-01,2019-12-31
Institut national des sciences et industries du vivant et de l'environnement,AGPT,2020-01-01,9999-12-31
École normale supérieure Paris-Saclay,DENS,2015-09-01,9999-12-31
École centrale des arts et manufactures,ECAP,2015-09-01,9999-12-31
École des hautes études commerciales de Paris,EHEC,2015-09-01,2019-08-31
"École supérieure de télégraphie, École nationale supérieure des télécommunications (ENST)",ENST,2015-09-01,2019-08-31
"École supérieure de télégraphie, École nationale supérieure des télécommunications (ENST)",ENST,2019-09-01,9999-12-31
École polytechnique,EPXX,2015-09-01,2019-08-31
École polytechnique,EPXX,2019-09-01,9999-12-31
École nationale supérieure de techniques avancées,ESTA,2015-09-01,2019-08-31


## Contrôle qualité des données

### Validité des identifiants (PPN)

In [8]:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

select * where { 
    ?ppn a foaf:Organization.
    
    FILTER (STRSTARTS(str(?ppn), "http://www.abes.fr/entities/organizations/ppn")) 
    FILTER (!regex(STR(?ppn), '/\\d{8}(\\d|X)', "i"))
}

ppn


### Validité des habilitations

* Seuls les établissements ayant une personnalité juridique peuvent délivrer le doctorat (que se passe-t-il s'ils changent de personnalité, cela implique-t-il une nouvelle habilitation ?)
* Les habilitations ne peuvent pas être antérieures ou postérieures à l'existence de l'établissement

In [14]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?codeEtab ?habilitation ?formeJuridique ?startHabilitation ?inception ?endHabilitation ?destruction ?diagnostic WHERE {
    {
        # On récupère les organisations qui sont ou on été habilités (c'est à dire qui ont un code établissement `?codeEtab`) ainsi que leur forme juridique.
        ?org a foaf:Organization;
             mov:codeEtabid ?codeEtab;
             wdt:P1454 ?formeJuridique.
        
        # On filtre les établissements qui ont explicitement une absence de personnalité juridique : `Sans personnalité juridique`.
        FILTER(?formeJuridique = "Sans personnalité juridique"@fr)
        
        # On assigne à la variable `?diagnostic` le contenu du message de diagnostic.
        BIND("Forme juridique invalide"@fr as ?diagnostic)
    } UNION {
        ?habilitation a mov:EtatDoctoral;
            mov:start ?startHabilitation.
        
        
        

        OPTIONAL {
            ?habilitation mov:end ?endHabilitation.
        }
        
        ?org mov:codeEtabid ?codeEtab.
        
        ?existence a mov:Existence;
        

                mov:from ?org.


        ?existence mov:start ?creation_etab.

        OPTIONAL {
            ?existence mov:end ?supression_etab.
        }
        
        # On filtre les établissement pour lesquels :
        # les dates d'habilitation dépassent les date d'existence
        # la fin de l'habilitation antérieur au début de l'habilitation
        FILTER(?startHabilitation < ?inception || ?endHabilitation > ?destruction || ?endHabilitation < ?startHabilitation)
        BIND("Dates d'habilitation invalides"@fr as ?diagnostic)
    }
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

codeEtab,habilitation,formeJuridique,startHabilitation,inception,endHabilitation,destruction,diagnostic
EMAC,,Sans personnalité juridique,,,,,Forme juridique invalide
EMAL,,Sans personnalité juridique,,,,,Forme juridique invalide
EMSE,,Sans personnalité juridique,,,,,Forme juridique invalide
ENIS,,Sans personnalité juridique,,,,,Forme juridique invalide
ENST,,Sans personnalité juridique,,,,,Forme juridique invalide
IMTA,,Sans personnalité juridique,,,,,Forme juridique invalide
INPL,,Sans personnalité juridique,,,,,Forme juridique invalide
NSAM,,Sans personnalité juridique,,,,,Forme juridique invalide
TELB,,Sans personnalité juridique,,,,,Forme juridique invalide


### Établissements non liés à Paysage

In [10]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?org_abes ?name WHERE {
    ?org_abes a foaf:Organization;
        foaf:name ?name.
    
    FILTER (STRSTARTS(str(?org_abes), "http://www.abes.fr/entities/organizations/ppn"))
    
    MINUS { 
        ?org_paysage (^owl:sameAs|owl:sameAs) ?org_abes .
        FILTER (STRSTARTS(str(?org_paysage), "http://paysage.mesri.fr/structures/"))
    }
    
    OPTIONAL {
        ?org_abes (^owl:sameAs|owl:sameAs) ?relation.    
    }
}

Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

org_abes,name
http://www.abes.fr/entities/organizations/ppn/026402882,Aix-Marseille 2
http://www.abes.fr/entities/organizations/ppn/028968832,Alger (Période coloniale)
http://www.abes.fr/entities/organizations/ppn/030142199,Bordeaux (1441-1970)
http://www.abes.fr/entities/organizations/ppn/030142865,Clermont-Ferrand
http://www.abes.fr/entities/organizations/ppn/034751386,Ecole nationale supérieure des industries alimentaires - Massy
http://www.abes.fr/entities/organizations/ppn/030937809,Ecole des Mines de Douai
http://www.abes.fr/entities/organizations/ppn/026436930,"Ecole nationale du génie rural, des eaux et forêts"
http://www.abes.fr/entities/organizations/ppn/026523477,Ecole nationale supérieure d'agronomie de Montpellier
http://www.abes.fr/entities/organizations/ppn/029884942,Faculté de droit et sciences économiques de Grenoble
http://www.abes.fr/entities/organizations/ppn/072323965,Faculté des lettres et sciences humaines de Grenoble


### États et existences incompatibles

Les états sont considérés comme invalides s'ils débutent avant la période d'existence de l'établissement ou s'ils cessent après cette période (à l'exception les états de succession).

In [11]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?state ?existence ?state_start ?state_end ?existence_start ?existence_end ?diagnostic WHERE {
    ?existence a <http://www.abes.fr/entities/etats>;
        mov:from ?from;
        mov:statetype "existence";
        mov:start ?existence_start.
    
    OPTIONAL {
        ?existence mov:end ?e_end.
    }
    
    OPTIONAL {
        ?state a <http://www.abes.fr/entities/etats>;
           mov:from ?from;
           mov:start ?state_start;
           mov:statetype ?state_type.
        
        OPTIONAL {
            ?state mov:end ?s_end.
        }
    }
    
    FILTER(?state_type NOT IN ("existence", "succession"))
    
    BIND (IF(!BOUND(?s_end), "9999-12-31"^^xsd:date, ?s_end )  AS ?state_end)
    BIND (IF(!BOUND(?e_end), "9999-12-31"^^xsd:date, ?e_end )  AS ?existence_end)
    
    BIND( IF(?state_start < ?existence_start && ?state_end > ?existence_end, "?state_start < ?existence_start && ?s_end > ?e_end", BNODE()) AS ?diagnostic)
    BIND( IF(?state_start < ?existence_start, "l'état débute avant l'existence", BNODE()) AS ?diagnostic)
    BIND( IF(?state_end > ?existence_end, "l'état termine après l'existence", BNODE()) AS ?diagnostic)

    FILTER(!ISBLANK(?diagnostic))
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

state,existence,state_start,state_end,existence_start,existence_end,diagnostic


## Enrichissement des données

### Candidats enrichissement des données ABES/Paysage
On recherche les candidats non liés par leur nom (foaf:name|skos:prefLabel|skos:altLabel)

In [12]:
%qparam infer false
%qparam sameAs false

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT * WHERE {
    ?org_abes a foaf:Organization;
        foaf:name ?name.
    
    FILTER (STRSTARTS(str(?org_abes), "http://www.abes.fr/entities/organizations/ppn"))
    
    MINUS { 
        ?org_paysage (^owl:sameAs|owl:sameAs) ?org_abes .
        FILTER (STRSTARTS(str(?org_paysage), "http://paysage.mesri.fr/structures/"))
    }
    
    ?candidat_paysage (foaf:name|skos:prefLabel|skos:altLabel) ?name.
    FILTER (STRSTARTS(str(?candidat_paysage), "http://paysage.mesri.fr/structures/"))
}

Param set: infer = false
Param set: sameAs = false

org_abes,name,org_paysage,candidat_paysage
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n


## Ontologies

<center>
    <figure>
        <img src="movies_ontology_v1.svg">
        <figcaption>Ontologie movies v1</figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src="movies_ontology_v2.svg">
        <figcaption>Ontologie movies v2</figcaption>
    </figure>
</center>

